<img src="images/dask_horizontal.svg" align="right" width="30%">

# Data Storage

<img src="images/hdd.jpg" width="20%" align="right">
Efficient storage can dramatically improve performance, particularly when operating repeatedly from disk.

Decompressing text and parsing CSV files is expensive.  One of the most effective strategies with medium data is to use a binary storage format like HDF5.  Often the performance gains from doing this is sufficient so that you can switch back to using Pandas again instead of using `dask.dataframe`.

In this section we'll learn how to efficiently arrange and store your datasets in on-disk binary formats.  We'll use the following:

1.  [Pandas `HDFStore`](http://pandas.pydata.org/pandas-docs/stable/io.html#io-hdf5) format on top of `HDF5`
2.  Categoricals for storing text data numerically

**Main Take-aways**

1.  Storage formats affect performance by an order of magnitude
2.  Text data will keep even a fast format like HDF5 slow
3.  A combination of binary formats, column storage, and partitioned data turns one second wait times into 80ms wait times.

## Setup

Create data if we don't have any

In [1]:
from prep import accounts_csvs
accounts_csvs(3, 1000000, 500)


## Read CSV

First we read our csv data as before.

CSV and other text-based file formats are the most common storage for data from many sources, because they require minimal pre-processing, can be written line-by-line and are human-readable. Since Pandas' `read_csv` is well-optimized, CSVs are a reasonable input, but far from optimized, since reading required extensive text parsing.

In [2]:
import os
filename = os.path.join('data', 'accounts.*.csv')
filename

'data/accounts.*.csv'

In [3]:
import dask.dataframe as dd

In [4]:
%%time
df_csv = dd.read_csv(filename)
df_csv.head()

CPU times: user 170 ms, sys: 50 ms, total: 220 ms
Wall time: 320 ms


,id,names,amount
0,358,Frank,453
1,227,Frank,-1348
2,103,Dan,7045
3,276,Norbert,1510
4,98,Jerry,2189


### Write to HDF5

HDF5 and netCDF are binary array formats very commonly used in the scientific realm.

Pandas contains a specialized HDF5 format, `HDFStore`.  The ``dd.DataFrame.to_hdf`` method works exactly like the ``pd.DataFrame.to_hdf`` method.

In [5]:
target = os.path.join('data', 'accounts.h5')
target

'data/accounts.h5'

In [6]:
# convert to binary format, takes some time up-front
%time df_csv.to_hdf(target, '/data')

CPU times: user 2.91 s, sys: 500 ms, total: 3.41 s
Wall time: 6.94 s


['data/accounts.h5', 'data/accounts.h5', 'data/accounts.h5']

In [7]:
%%time
# same data as before
df_hdf = dd.read_hdf(target, '/data')
df_hdf.head()

CPU times: user 550 ms, sys: 100 ms, total: 650 ms
Wall time: 766 ms


,id,names,amount
0,358,Frank,453
1,227,Frank,-1348
2,103,Dan,7045
3,276,Norbert,1510
4,98,Jerry,2189


### Compare CSV to HDF5 speeds

We do a simple computation that requires reading a column of our dataset and compare performance between CSV files and our newly created HDF5 file.  Which do you expect to be faster?

In [8]:
%time df_csv.amount.sum().compute()

CPU times: user 580 ms, sys: 530 ms, total: 1.11 s
Wall time: 546 ms


2833767903

In [9]:
%time df_hdf.amount.sum().compute()

CPU times: user 1.67 s, sys: 310 ms, total: 1.98 s
Wall time: 2.21 s


2833767903

Sadly they are about the same, or perhaps even slower. 

The culprit here is `names` column, which is of `object` dtype and thus hard to store efficiently.  There are two problems here:

1.  How do we store text data like `names` efficiently on disk?
2.  Why did we have to read the `names` column when all we wanted was `amount`

### 1.  Store text efficiently with categoricals

We can use Pandas categoricals to replace our object dtypes with a numerical representation.  This takes a bit more time up front, but results in better performance.

More on categoricals at the [pandas docs](http://pandas.pydata.org/pandas-docs/stable/categorical.html) and [this blogpost](http://matthewrocklin.com/blog/work/2015/06/18/Categoricals).

In [10]:
# Categorize data, then store in HDFStore
%time df_hdf.categorize(columns=['names']).to_hdf(target, '/data2')

CPU times: user 5.67 s, sys: 820 ms, total: 6.49 s
Wall time: 10.4 s


['data/accounts.h5', 'data/accounts.h5', 'data/accounts.h5']

In [11]:
# It looks the same
df_hdf = dd.read_hdf(target, '/data2')
df_hdf.head()

,id,names,amount
0,358,Frank,453
1,227,Frank,-1348
2,103,Dan,7045
3,276,Norbert,1510
4,98,Jerry,2189


In [12]:
# But loads more quickly
%time df_hdf.amount.sum().compute()

CPU times: user 150 ms, sys: 80 ms, total: 230 ms
Wall time: 413 ms


2833767903

This is now definitely faster than before.  This tells us that it's not only the file type that we use but also how we represent our variables that influences storage performance. 

How does the performance of reading depend on the scheduler we use? You can try this with threaded, processes and distributed.

However this can still be better.  We had to read all of the columns (`names` and `amount`) in order to compute the sum of one (`amount`).  We'll improve further on this with `parquet`, an on-disk column-store.  First though we learn about how to set an index in a dask.dataframe.

### Exercise

`fastparquet` is a library for interacting with parquet-format files, which are a very common format in the Big Data ecosystem, and used by tools such as Hadoop, Spark and Impala.

In [13]:
target = os.path.join('data', 'accounts.parquet')
df_csv.categorize(columns=['names']).to_parquet(target, storage_options={"has_nulls": True}, engine="fastparquet")

Investigate the file structure in the resultant new directory - what do you suppose those files are for?

`to_parquet` comes with many options, such as compression, whether to explicitly write NULLs information (not necessary in this case), and how to encode strings. You can experiment with these, to see what effect they have on the file size and the processing times, below.

In [14]:
ls -l data/accounts.parquet/

total 74120
-rw-r--r-- 1 jovyan users      723 Oct  5 23:06 _common_metadata
-rw-r--r-- 1 jovyan users     1977 Oct  5 23:06 _metadata
-rw-r--r-- 1 jovyan users 25001853 Oct  5 23:06 part.0.parquet
-rw-r--r-- 1 jovyan users 25001853 Oct  5 23:06 part.1.parquet
-rw-r--r-- 1 jovyan users 25001853 Oct  5 23:06 part.2.parquet


In [15]:
df_p = dd.read_parquet(target)
# note that column names shows the type of the values - we could
# choose to load as a categorical column or not.
df_p.dtypes

id           int64
names     category
amount       int64
dtype: object

Rerun the sum computation above for this version of the data, and time how long it takes. You may want to try this more than once - it is common for many libraries to do various setup work when called for the first time.

In [16]:
%time df_p.amount.sum().compute()

CPU times: user 400 ms, sys: 40 ms, total: 440 ms
Wall time: 476 ms


2833767903

When archiving data, it is common to sort and partition by a column with unique identifiers, to facilitate fast look-ups later. For this data, that column is `id`. Time how long it takes to retrieve the rows corresponding to `id==100` from the raw CSV, from HDF5 and parquet versions, and finally from a new parquet version written after applying `set_index('id')`.

In [17]:
target_new = os.path.join('data', 'accounts_new.parquet')
df_p.set_index('id').to_parquet(target_new, storage_options={"has_nulls": True}, engine="fastparquet")

In [26]:
df_p_new = dd.read_parquet(target_new)

In [27]:
df_p_new.head()

,names,amount
id,,
0,Norbert,475
0,Norbert,392
0,Norbert,464
0,Norbert,495
0,Norbert,442


In [34]:
(df_p_new.loc[100]).compute()

,names,amount
id,,
100,Victor,51
100,Victor,60
100,Victor,58
100,Victor,50
100,Victor,51
...,...,...
100,Victor,54
100,Victor,55
100,Victor,57


In [35]:
%time results_csv = df_csv[df_csv['id'] == 100].compute()
%time results_hdf = df_hdf[df_hdf['id'] == 100].compute()
%time results_p = df_p[df_p['id'] == 100].compute()
%time results_p_new = df_p_new.loc[100].compute()

CPU times: user 440 ms, sys: 120 ms, total: 560 ms
Wall time: 358 ms
CPU times: user 130 ms, sys: 70 ms, total: 200 ms
Wall time: 317 ms
CPU times: user 50 ms, sys: 50 ms, total: 100 ms
Wall time: 132 ms
CPU times: user 20 ms, sys: 10 ms, total: 30 ms
Wall time: 52.9 ms


## Remote files

Dask can access various cloud- and cluster-oriented data storage services such as Amazon S3 or HDFS

Advantages:
* scalable, secure storage

Disadvantages:
* network speed becomes bottleneck


The way to set up dataframes (and other collections) remains very similar to before. Note that the data here is available anonymously, but in general an extra parameter `storage_options=` can be passed with further details about how to interact with the remote storage.

```python
taxi = dd.read_csv('s3://nyc-tlc/trip data/yellow_tripdata_2015-*.csv',
                   storage_options={'anon': True})
```

**Warning**: operations over the Internet can take a long time to run. Such operations work really well in a cloud clustered set-up, e.g., amazon EC2 machines reading from S3 or Google compute machines reading from GCS.